In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import DepthwiseConv2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, InputLayer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define paths
train_dir = "D:/VIT Chennai/SEM 7/Capstone/ecg full/train"
test_dir = "D:/VIT Chennai/SEM 7/Capstone/ecg full/test"

# Constants
IMG_WIDTH, IMG_HEIGHT = 224, 159
BATCH_SIZE = 32
NUM_CLASSES = 4  

# Enhanced Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation')

# Custom Model Architecture
model = Sequential([
    InputLayer(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
    
    # Block 1
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    DepthwiseConv2D((3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    
    # Block 2
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    DepthwiseConv2D((3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    
    # Block 3
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    DepthwiseConv2D((3, 3), activation='relu', padding='same'),
    GlobalAveragePooling2D(),
    Dropout(0.4),
    
    # Classifier
    Dense(4, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0003),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Callbacks for Early Stopping and Saving the Best Model
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True)
]

# Train the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=50,  
    callbacks=callbacks,
    verbose=2)

# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

evaluation = model.evaluate(test_generator)
print(f'Test Loss: {evaluation[0]}')
print(f'Test Accuracy: {evaluation[1]}')


Found 745 images belonging to 4 classes.
Found 183 images belonging to 4 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 159, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 224, 159, 32)      128       
 Normalization)                                                  
                                                                 
 depthwise_conv2d (Depthwis  (None, 224, 159, 32)      320       
 eConv2D)                                                        
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 79, 32)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 112,

C:\Users\Rajat\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


23/23 - 45s - loss: 1.3955 - accuracy: 0.3086 - val_loss: 1.3841 - val_accuracy: 0.3125 - 45s/epoch - 2s/step
Epoch 3/50
23/23 - 42s - loss: 1.3839 - accuracy: 0.3072 - val_loss: 1.3835 - val_accuracy: 0.3187 - 42s/epoch - 2s/step
Epoch 4/50
23/23 - 43s - loss: 1.3827 - accuracy: 0.3058 - val_loss: 1.3833 - val_accuracy: 0.3063 - 43s/epoch - 2s/step
Epoch 5/50
23/23 - 47s - loss: 1.3807 - accuracy: 0.3086 - val_loss: 1.3814 - val_accuracy: 0.3063 - 47s/epoch - 2s/step
Epoch 6/50
23/23 - 42s - loss: 1.3827 - accuracy: 0.3114 - val_loss: 1.3803 - val_accuracy: 0.3125 - 42s/epoch - 2s/step
Epoch 7/50
23/23 - 42s - loss: 1.3813 - accuracy: 0.3072 - val_loss: 1.3807 - val_accuracy: 0.3125 - 42s/epoch - 2s/step
Epoch 8/50
23/23 - 40s - loss: 1.3775 - accuracy: 0.3100 - val_loss: 1.3815 - val_accuracy: 0.3000 - 40s/epoch - 2s/step
Epoch 9/50
23/23 - 41s - loss: 1.3827 - accuracy: 0.3058 - val_loss: 1.3789 - val_accuracy: 0.3000 - 41s/epoch - 2s/step
Epoch 10/50
